In [7]:
import json
import re

In [18]:
flatten = lambda _list: [item for sub_list in _list for item in sub_list]

In [6]:
with open("../../data/compound_train.json", 'r', encoding='utf-8') as f:
    train = json.load(f)['entry']

In [20]:
total = len(flatten([entry['Attributes']['原材料'] for entry in train]))
exist = 0
for entry in train:
    materials = entry['Attributes']['原材料']
    production = ''.join(entry['Attributes']['製造方法'])
    
    for material in materials:
        if re.search(re.escape(material), production):
            exist += 1
            
print("{}/{}".format(exist, total))

1114/1303
